In [1]:
from pathlib import Path   
import geopandas as gpd
import pandas as pd
from shapely import wkt
import pandas as pd
import pandas as pd
from datetime import datetime
import os
from shapely import wkt

import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
import pandas as pd

output_directory = "data/"
csv_files = [file for file in os.listdir(output_directory) if file.endswith("_final_geom.csv")]

# Initialize an empty list to store the data frames
data_frames = []

# Iterate through the .csv files and read them into data frames
for file in csv_files:
    file_path = os.path.join(output_directory, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Concatenate the data frames vertically
combined_df = pd.concat(data_frames, axis=0)

# Reset the index of the combined data frame
combined_df.reset_index(drop=True, inplace=True)


In [5]:
combined_df.head(5)

,h3_hexagon,geometry,tornado_count,mag_h3_history,inj_h3_history,fat_h3_history,loss_h3_history,closs_h3_history,len_h3_history,wid_h3_history,...,sum_years_diff,sum_property_loss,total_loss,total_years,avg_loss_per_year,avg_frequency,avg_property_loss,avg_mag_freq,names,date_y


In [2]:
import pandas as pd
import statsmodels.api as sm

# Load tornado event and property damage data
tornado_data = pd.read_csv(' tornado/notebooks/tornado_stats_final.csv')

In [3]:
tornado_data.head(3)

,h3_id,geometry,tornado_count,magnitude_history,injuries_history,fatality_history,property_loss_history,crop_loss_history,t_length_history,t_width_history,...,years_diff,num_events,sum_years_diff,sum_property_loss,total_loss,avg_loss_per_year,avg_frequency,avg_property_loss,avg_mag_freq,state
0,89441a91b9bffff,POLYGON ((-82.66624444281332 27.76590057338288...,21,0.29,6.0,0.0,66.0,0.0,1.23,38.71,...,"[0.394, 5.936, 0.222, 0.912, 0.854, 2.439, 0.0...",21,17.98,66.0,66.0,3.14,1.17,3.14,1.17,FL
1,89441a91bd7ffff,POLYGON ((-82.66307109133983 27.76672514389528...,21,0.29,6.0,0.0,66.0,0.0,1.23,38.71,...,"[0.394, 5.936, 0.222, 0.912, 0.854, 2.439, 0.0...",21,17.98,66.0,66.0,3.14,1.17,3.14,1.17,FL
2,89441a91867ffff,POLYGON ((-82.66707030598505 27.76283958002425...,21,0.29,6.0,0.0,66.0,0.0,1.23,38.71,...,"[0.394, 5.936, 0.222, 0.912, 0.854, 2.439, 0.0...",21,17.98,66.0,66.0,3.14,1.17,3.14,1.17,FL


In [ ]:
# Calculate the avg_frequency for each hexagon
#avg_frequency = tornado_data.groupby('h3_hexagon').size() / tornado_data['years_diff'].nunique()

# Calculate the total property damage for each hexagon
#total_damage = tornado_data.groupby('h3_hexagon')['property_damage'].sum()

# Create a new DataFrame with both variables
data = pd.DataFrame({'avg_frequency': avg_frequency, 'total_damage': total_damage})

# Run a regression analysis to determine the relationship between the variables
X = data['avg_frequency']
Y = data['total_damage']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()

# Print the results of the regression analysis
print(model.summary())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Load tornado event and property damage data
tornado_data = pd.read_csv('tornado_data.csv')

# Calculate the avg_frequency for each hexagon
avg_frequency = tornado_data.groupby('h3_hexagon').size() / tornado_data['years_diff'].nunique()

# Calculate the total property damage for each hexagon
total_damage = tornado_data.groupby('h3_hexagon')['property_damage'].sum()

# Create a new DataFrame with both variables
data = pd.DataFrame({'avg_frequency': avg_frequency, 'total_damage': total_damage})

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data[['avg_frequency']], data['total_damage'], test_size=0.2)

# Fit a random forest regression model to the training data
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Use the model to make predictions on the test data
predictions = model.predict(X_test)

# Print the mean squared error of the predictions
mse = ((predictions - y_test) ** 2).mean()
print('Mean Squared Error:', mse)
